### Data Science Essentials 4

**TN Med Helper** is a fictional company whose mission is to ensure access to healthcare for all Tennesseans. TN Med Helper has approached your data science consultancy for help identifying communities in Tennessee that need the most help in expanding access to healthcare.

In this project, we will use the [Medicare Disparities](https://data.cms.gov/mapping-medicare-disparities) data as a starting point for identifying such communities. Specifically, you will be provided with datasets containing the percent of Medicare beneficiaries who had an annual wellness visit (annual_wellness.csv), the number of all-cause hospitilizations per 1000 beneficiaries (hospitalizations.csv), and the number of emergency department visits per 1000 beneficiaries (emergency_department.csv). Over the next 8 weeks, you will work towards addressing the following three objectives.

First, TN Med Helper is concerned about communities either lacking access to healthcare or losing access to healthcare. They are looking to expand telehealth technologies into the vulnerable communities, and need your help to priortize areas most needing attention. your first objective is to identify which counties in Tennessee have the most severe lack of access to healthcare (either due to lack of hospitals, physicians, or both). Once you have identified these counties, see if you can find any common demographic or economic characteristics for these areas.

Second, TN Med Helper is interested in reducing the number of potentially preventable hospitalizations. Do areas that lack access to healthcare tend to have higher rates of emergency department visits or hospitalizations? Is there an association between the percentage of beneficiaries who had an annual wellness visit and rate of hospitalizations or emergency department visits?

In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [80]:
# Read in data files for Question 1
physicians = pd.read_csv('../data/primary_care_physicians.csv')
physicians = physicians.loc[physicians['state'] == 'Tennessee']
population = pd.read_csv('../data/population_by_county.csv')
population = population.loc[population['state'] == 'TENNESSEE']
hospitals = pd.read_csv('../data/Hospitals.csv')
hospitals = hospitals.loc[hospitals['STATE'] == 'TN']
unemployment = pd.read_csv('../data/tn_unemployment.csv')

# Read in data files for Question 2
wellness = pd.read_csv('../data/Medicare_Disparities_by_Population/annual_wellness.csv')
wellness = wellness.loc[wellness['state'] == 'TENNESSEE']
hospitalizations = pd.read_csv('../data/Medicare_Disparities_by_Population/hospitalizations.csv')
hospitalizations = hospitalizations.loc[hospitalizations['state'] == 'TENNESSEE']
er_visits = pd.read_csv('../data/Medicare_Disparities_by_Population/emergency_department.csv')
er_visits = er_visits.loc[er_visits['state'] == 'TENNESSEE']

Identify which counties in Tennessee have the most severe lack of access to healthcare (either due to lack of hospitals, physicians, or both)

In [ ]:
# Correct column variables and merge data files; create new coluumn for residents per pcp
population['county'] = population['county'].str.lower() # make all county names lower case
population['county'] = population['county'].str.split(' county', expand = True)[0]

physicians['county'] = physicians['county'].str.lower() # make all county names lower case

physicians = pd.merge(left = physicians,
         right = population[['county','population', 'urban']])
physicians['residents_per_pcp'] = physicians['population'] / physicians['primary_care_physicians']

# Create categories for level of care
# Use .loc to add column, picks the new shadac category based on value in 'residents_per_pcp'
physicians.loc[physicians['residents_per_pcp'] < 1500, 'shadac_category'] = 'adequate'
physicians.loc[(physicians['residents_per_pcp'] >= 1500) & 
       (physicians['residents_per_pcp'] < 3500), 'shadac_category'] = 'moderately inadequate'
physicians.loc[(physicians['residents_per_pcp'] >= 3500), 'shadac_category'] = 'low inadequate'

In [82]:
# Return counties where shadac category is low inadequate for residents_per_pcp
Lack_of_Physicians = physicians.loc[(physicians['shadac_category'] == 'low inadequate')]
Lack_of_Physicians['county']

2         benton
3        bledsoe
7         cannon
11       chester
13          clay
16      crockett
28      grainger
30        grundy
33       hancock
34      hardeman
36       hawkins
37       haywood
38     henderson
40       hickman
41       houston
42     humphreys
43       jackson
47          lake
48    lauderdale
50         lewis
55         macon
58      marshall
61        monroe
63         moore
64        morgan
67         perry
80       stewart
83        tipton
84     trousdale
86         union
87     van buren
Name: county, dtype: object

In [90]:
# Merge hospitals and physicians data frames
# Correct column variables and merge data files; create new coluumn for residents per hospital
hospitals['COUNTY'] = hospitals['COUNTY'].str.lower() # make all county names lower case

physicians = pd.merge(left = physicians,
         right = hospitals[['COUNTY','NAME']].rename(columns = {'COUNTY': 'county', 'NAME': 'hospital'}))
physicians.head()

,FIPS,state,county,primary_care_physicians,population,urban,residents_per_pcp,shadac_category,NAME,hospital
0,47001,Tennessee,anderson,39.0,76061,Urban,1950.282051,moderately inadequate,METHODIST MEDICAL CENTER OF OAK RIDGE,METHODIST MEDICAL CENTER OF OAK RIDGE
1,47003,Tennessee,bedford,15.0,48292,Rural,3219.466667,moderately inadequate,BEDFORD COUNTY MEDICAL CENTER,BEDFORD COUNTY MEDICAL CENTER
2,47003,Tennessee,bedford,15.0,48292,Rural,3219.466667,moderately inadequate,BEDFORD COUNTY MEDICAL CENTER,TENNOVA HEALTHCARE - SHELBYVILLE
3,47003,Tennessee,bedford,15.0,48292,Rural,3219.466667,moderately inadequate,TENNOVA HEALTHCARE - SHELBYVILLE,BEDFORD COUNTY MEDICAL CENTER
4,47003,Tennessee,bedford,15.0,48292,Rural,3219.466667,moderately inadequate,TENNOVA HEALTHCARE - SHELBYVILLE,TENNOVA HEALTHCARE - SHELBYVILLE


In [87]:
# Find the number of hospitals per county 


In [91]:
# Identify which counties in Tennessee have the most severe lack of access to healthcare 
# due to lack of hospitals & physicians

Once you have identified these counties, see if you can find any common demographic or economic characteristics for these areas.

In [92]:
# Work with unemployment file
unemployment['Name'] = unemployment['Name'].str.lower()
unemployment['Name'] = unemployment['Name'].str.split(' county', expand = True)[0]
physicians = pd.merge(left = physicians,
         right = unemployment[['Name', 'unemployment_rate']].rename(columns = {'Name': 'county'}))

NameError: name 'unemployment' is not defined